In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650'

In [11]:
x = torch.randint(1, 100, (100, 100))

In [12]:
print(x.device)

cpu


In [13]:
res_cpu = x ** 2

In [14]:
x = x.to(torch.device('cuda'))

In [15]:
print(x.device)

cuda:0


In [9]:
res_gpu = x ** 2

In [10]:
assert torch.equal(res_cpu, res_gpu.cpu())

In [15]:
%cd content


/home/ivan/Documents/Code/opencv/content


In [16]:
!git clone https://github.com/opencv/opencv

Cloning into 'opencv'...
remote: Enumerating objects: 311256, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 311256 (delta 13), reused 55 (delta 7), pack-reused 311182
Receiving objects: 100% (311256/311256), 499.01 MiB | 6.87 MiB/s, done.
Resolving deltas: 100% (216617/216617), done.


In [17]:
!git clone https://github.com/opencv/opencv_contrib

Cloning into 'opencv_contrib'...
remote: Enumerating objects: 38017, done.
remote: Counting objects: 100% (296/296), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 38017 (delta 151), reused 213 (delta 112), pack-reused 37721
Receiving objects: 100% (38017/38017), 133.36 MiB | 5.48 MiB/s, done.
Resolving deltas: 100% (23618/23618), done.


In [20]:
!mkdir build

/home/ivan/Documents/Code/opencv/content/build


In [22]:
!cmake -DOPENCV_EXTRA_MODULES_PATH=content/opencv_contrib/modules  -DBUILD_SHARED_LIBS=OFF  -DBUILD_TESTS=OFF  -DBUILD_PERF_TESTS=OFF -DBUILD_EXAMPLES=OFF -DWITH_OPENEXR=OFF -DWITH_CUDA=ON -DWITH_CUBLAS=ON -DWITH_CUDNN=ON -DOPENCV_DNN_CUDA=ON content/opencv

CMake Error: The source directory "/home/ivan/Documents/Code/opencv/content/build/content/opencv" does not exist.
Specify --help for usage, or press the help button on the CMake GUI.


In [14]:
!cmake -DOPENCV_EXTRA_MODULES_PATH=content/opencv_contrib/modules  -DBUILD_SHARED_LIBS=OFF  -DBUILD_TESTS=OFF  -DBUILD_PERF_TESTS=OFF -DBUILD_EXAMPLES=OFF -DWITH_OPENEXR=OFF -DWITH_CUDA=ON -DWITH_CUBLAS=ON -DWITH_CUDNN=ON -DOPENCV_DNN_CUDA=ON content/opencv

CMake Error: The source directory "/home/ivan/Documents/Code/opencv/content/opencv" does not exist.
Specify --help for usage, or press the help button on the CMake GUI.
